In [51]:
import pandas as pd
from collections import defaultdict
from datetime import datetime
import numpy as np
import pandas as pd

In [56]:
biased_complement = pd.read_excel('data/twitter_data/twitter_no_label_9_19_17.xlsx')

In [67]:
def normalize(df):
    return df.groupby(['source_id']).aggregate(
        {'label_someone_got_sick':'first',
         'text':'first',
         'created_date':'first',
        'twitter_user_id': 'first',
        'location': 'first'}).reset_index()

def rename(g, is_foodborne_map={'YES': 1, 'NO': 0}):
    g.loc[:, 'url'] = 'https://twitter.com/' + g.twitter_user_id.astype(str) + '/status/' + g.source_id.astype(str)
    g = g.filter(['created_date', 'text', 'url', 'location', 'label_someone_got_sick']).rename(columns={
    'label_someone_got_sick': 'is_foodborne',
    'created_date': 'date'})
    g.is_foodborne = g.is_foodborne.map(is_foodborne_map)
    g.loc[:,'is_foodborne'] = ''
    g.loc[:,'is_multiple'] = ''
    g.loc[:,'is_nyc_located'] = ''
    return g

def validate(g):
    return g[~g.is_foodborne.isnull()]

def split(df, date):
    return (df[df.date < date], df[df.date >= date])


In [68]:
biased_complement_csv = validate(rename(normalize(biased_complement), is_foodborne_map=defaultdict(int)))

In [69]:
test_split_date_str = '6/1/2017'
test_split_date = datetime.strptime(test_split_date_str, '%m/%d/%Y')
(old_biased_complement, new_biased_complement) = split(biased_complement_csv, test_split_date)

In [70]:
print len(old_biased_complement)
print len(new_biased_complement)


23115
6885


In [71]:
old_biased_complement_sampled = old_biased_complement.sample(n=1000, random_state=0)
new_biased_complement_sampled = new_biased_complement.sample(n=1000, random_state=0)

In [72]:
old_biased_complement_sampled.to_csv('data/twitter_data/old_biased_complement_sampled.csv', encoding='UTF-8')
new_biased_complement_sampled.to_csv('data/twitter_data/new_biased_complement_sampled.csv', encoding='UTF-8')


In [23]:
from experiments.baseline_experiment_util import setup_baseline_data

In [26]:
data = setup_baseline_data(dataset='yelp', train_regime='gold', test_regime='gold', data_path='data/yelp_data/')

experiments/baseline_experiment_util.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  old_biased['is_foodborne'] = old_biased['is_foodborne'].map({'Yes':1, 'No':0})
experiments/baseline_experiment_util.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  new_biased['is_foodborne'] = new_biased['is_foodborne'].map({'Yes':1, 'No':0})
experiments/baseline_experiment_util.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [27]:
data.keys()

['test_data', 'U', 'train_data']

In [28]:
data['test_data'].keys()

['is_multiple', 'text', 'is_foodborne', 'is_biased']

In [37]:
pd.Series(data['test_data']['is_biased']).value_counts()

True     1975
False    1000
dtype: int64

In [42]:
biased_complement.head(2).fillna("").to_dict(orient="records")

[{u'action_item': '',
  u'business_id': '',
  u'callDate': '',
  u'complaint_date': '',
  u'complaint_num': '',
  u'conversation_tracking_attempted_': Timestamp('1900-01-02 00:00:00'),
  u'created_date': Timestamp('2016-11-28 12:06:08.380000'),
  u'document_id': '',
  u'dohmh_id': '',
  u'emailDate': '',
  u'final_status': '',
  u'final_status_date': '',
  u'foursquare_checkin_attempted_dat': Timestamp('1900-01-02 00:00:00'),
  u'id': 867682,
  u'in_reply_to_tweet_id': '',
  u'incident_date': '',
  u'incident_description': '',
  u'interviewDate': '',
  u'interviewStatus': '',
  u'is311': '',
  u'is_foodborne': '',
  u'is_nyc': '',
  u'is_nycetrack': '',
  u'is_same_as_reviewer': '',
  u'label_notes': '',
  u'label_nyc_located': u'UNKNOWN',
  u'label_serious_enough': '',
  u'label_someone_got_sick': '',
  u'label_timing_consistent': '',
  u'labeled_by_user_id': '',
  u'labeled_date': Timestamp('1900-01-02 00:00:00'),
  u'latitude': 0.0,
  u'linkSentDate': '',
  u'location': '',
  u'long

In [50]:
print [x for x in list(biased_complement.columns.astype(str)) if 'user' in x or 'loc' in x]
print biased_complement.location.value_counts().head()
print biased_complement.label_nyc_located.value_counts().head()

['labeled_by_user_id', 'label_nyc_located', 'user_tweet_id', 'twitter_user_id', 'location', 'user_id']
United States      339
Los Angeles, CA    120
California, USA    107
butt                92
Chicago, IL         91
Name: location, dtype: int64
UNKNOWN    30000
Name: label_nyc_located, dtype: int64


In [73]:
biased_complement_csv.head(10)

,date,text,url,location,is_foodborne,is_multiple,is_nyc_located
0,2016-11-28 12:06:08.380,"""""""You think I give a damn about a Grammy? Hal...",https://twitter.com/316179846/status/801291697...,NaN,,,
1,2016-11-28 12:09:43.267,But that hit in the stomach ???? @msleamichele,https://twitter.com/3282891990/status/80129675...,NaN,,,
2,2016-11-28 12:09:44.720,"@_nybarra knees weak, arms are heavy: there's ...",https://twitter.com/2915226278/status/80129678...,NaN,,,
3,2016-11-28 12:11:52.983,My stomach hurt y'all,https://twitter.com/796757687300227077/status/...,"Tennessee, USA",,,
4,2016-11-28 12:15:12.827,"Stomach is growling, time for some grub!!",https://twitter.com/1870056282/status/80130553...,NaN,,,
5,2016-11-28 12:20:26.967,@GreggsOfficial #foodpoisoning #avoid\nThanks ...,https://twitter.com/591374301/status/801314485...,Monaco-worldwide,,,
6,2016-11-28 12:20:40.340,"Seriously, I need that vomit emogee... https:/...",https://twitter.com/2817164700/status/80131486...,"Vancouver, British Columbia",,,
7,2016-11-28 12:21:02.123,"my stomach hurts. i'm stressed, i'm moody and ...",https://twitter.com/775125243660820481/status/...,NaN,,,
8,2016-11-28 12:21:37.183,Definitely a rest day for me. Stomach acting u...,https://twitter.com/3888296714/status/80131653...,NaN,,,
9,2016-11-28 12:22:52.217,Crying at a sappy Hallmark Christmas movie whi...,https://twitter.com/449182873/status/801319215...,NaN,,,
